# Cloud Pak for Data Virtualization Demonstration
This notebook will connect to a CP4D cluster and demonstrate how SQL can be run against a variety of data sources without requiring any knowledge of how these data sources are accessed.

The first step is to load the Db2 extensions that allow us to run Db2 commands directly against CP4D.

In [1]:
%run db2.ipynb

Db2 Extensions Loaded.


A connection to the database is required before we can run any SQL statements. Run the following statement to connect to the Cloud Pak for Data cluster. **Note:** If you need to change the connection information, edit the Connection document which is found in the table of contents.

In [2]:
%run connection.ipynb

Connection successful.


## Stock Symbol Table
### Get information about the stocks that are in the database
**Db2 Warehouse on CPD**

In [3]:
%sql -a select * from TRADING.STOCK_SYMBOLS FETCH FIRST 10 ROWS ONLY

,COMPANY,SYMBOL
0,Exxon Mobil Corporation,XOM
1,DowDuPont Inc.,DWDP
2,The Travelers Companies Inc.,TRV
3,International Business Machines Corporation,IBM
4,NIKE Inc.,NKE
5,Chevron Corporation,CVX
6,United Technologies Corporation,UTX
7,Walgreens Boots Alliance Inc.,WBA
8,JPMorgan Chase & Co.,JPM
9,Microsoft Corporation,MSFT


## Top Buy/Sell By Customer (All Stocks)
**AWS - Db2, EDB on PREM**

In [4]:
%%sql 
WITH
    TX_DAY(TX_DATE) AS (
      VALUES NEXT_DAY(DATE('2017-12-18') + INT(RAND()*350) DAYS, 'Thursday')
    ),
    PURCHASED(CUSTID,AMOUNT) AS (
      SELECT CUSTID, SUM(QUANTITY) AS AMOUNT
        FROM TRADING.STOCK_TRANSACTIONS ST, TX_DAY TX
      WHERE ST.TX_DATE = TX.TX_DATE
      GROUP BY CUSTID
    )
    SELECT C.LASTNAME, C.CITY, C.STATE, P.AMOUNT
      FROM PURCHASED P, TRADING.CUSTOMER C
      WHERE C.CUSTID = P.CUSTID
    ORDER BY ABS(P.AMOUNT) DESC
    FETCH FIRST 9 ROWS ONLY 

,LASTNAME,CITY,STATE,AMOUNT
0,Erickson,Columbia,MO,132
1,Collier,Butte,MT,128
2,Graves,Rockford,IL,120
3,Humphrey,White Plains,NY,118
4,Oconnor,Aiken,SC,115
5,Berger,Sacramento,CA,113
6,Hendricks,Waco,TX,112
7,Lott,Hicksville,NY,110
8,Burch,Acton,ME,109


## Stock History Table
### Get Price of a Stock over the Year
Set the Stock Symbol in the line below.

**Virtual CSV**

In [5]:
stock = 'INTC'

In [6]:
%%sql 
SELECT TX_DATE, WEEK(TX_DATE) AS WEEK FROM TRADING.STOCK_HISTORY
FETCH FIRST 10 ROWS ONLY

,TX_DATE,WEEK
0,2017-12-18,51
1,2017-12-19,51
2,2017-12-20,51
3,2017-12-21,51
4,2017-12-22,51
5,2017-12-26,52
6,2017-12-27,52
7,2017-12-28,52
8,2017-12-29,52
9,2018-01-02,1


In [7]:
%%sql 
SELECT WEEK(TX_DATE) AS WEEKNUM FROM TRADING.STOCK_HISTORY
WHERE TX_DATE != '2017-12-02' AND SYMBOL = 'INTC'
FETCH FIRST 10 ROWS ONLY

,WEEKNUM
0,51
1,51
2,51
3,51
4,51
5,52
6,52
7,52
8,52
9,1


In [8]:
%%sql
WITH BOUGHT(SYMBOL, AMOUNT) AS
  (
  SELECT SYMBOL, SUM(QUANTITY) FROM TRADING.STOCK_TRANSACTIONS
  WHERE QUANTITY > 0
  GROUP BY SYMBOL
  ),
SOLD(SYMBOL, AMOUNT) AS
  (
  SELECT SYMBOL, -SUM(QUANTITY) FROM TRADING.STOCK_TRANSACTIONS
  WHERE QUANTITY < 0
  GROUP BY SYMBOL
  )
SELECT B.SYMBOL, B.AMOUNT AS BOUGHT, S.AMOUNT AS SOLD
FROM BOUGHT B, SOLD S
WHERE B.SYMBOL = S.SYMBOL
ORDER BY B.AMOUNT DESC
FETCH FIRST 5 ROWS ONLY

,SYMBOL,BOUGHT,SOLD
0,KO,694562,409390
1,CSCO,692836,408547
2,PFE,683095,393959
3,INTC,639324,368778
4,VZ,592401,340565


## Customer Accounts
### Show Top 5 Customer Balance
**AWS - Db2, Azure - Db2, Postgres**

In [9]:
%%sql
SELECT CUSTID, BALANCE FROM TRADING.ACCOUNTS
ORDER BY BALANCE DESC
FETCH FIRST 5 ROWS ONLY

,CUSTID,BALANCE
0,103778,80400.22
1,105969,78716.35
2,102949,78433.12
3,108130,75311.06
4,108045,72929.54


### Show Bottom 5 Customer Balance
**AWS - Db2, Azure - Db2, Postgres**

In [10]:
%%sql
SELECT CUSTID, BALANCE FROM TRADING.ACCOUNTS
ORDER BY BALANCE ASC
FETCH FIRST 5 ROWS ONLY

,CUSTID,BALANCE
0,101744,-102071.20
1,101744,-102071.20
2,103556,-95672.61
3,100700,-88780.18
4,100700,-88780.18


## Selecting Customer Information from MongoDB
The MongoDB database has customer information in a document format. In order to materialize the document data as relational tables, a total of four virtual tables are generated. The following query shows the tables that are generated for the Customer document collection.

In [11]:
%sql select * from syscat.tables where TABSCHEMA = 'MONGOONCPD' and TABNAME like 'CUSTOMER%'

,TABSCHEMA,TABNAME,OWNER,OWNERTYPE,TYPE,STATUS,BASE_TABSCHEMA,BASE_TABNAME,ROWTYPESCHEMA,ROWTYPENAME,CREATE_TIME,ALTER_TIME,INVALIDATE_TIME,STATS_TIME,COLCOUNT,TABLEID,TBSPACEID,CARD,NPAGES,MPAGES,FPAGES,NPARTITIONS,NFILES,TABLESIZE,OVERFLOW,TBSPACE,INDEX_TBSPACE,LONG_TBSPACE,PARENTS,CHILDREN,SELFREFS,KEYCOLUMNS,KEYINDEXID,KEYUNIQUE,CHECKCOUNT,DATACAPTURE,CONST_CHECKED,PMAP_ID,PARTITION_MODE,LOG_ATTRIBUTE,PCTFREE,APPEND_MODE,REFRESH,REFRESH_TIME,LOCKSIZE,VOLATILE,ROW_FORMAT,PROPERTY,STATISTICS_PROFILE,COMPRESSION,ROWCOMPMODE,ACCESS_MODE,CLUSTERED,ACTIVE_BLOCKS,DROPRULE,MAXFREESPACESEARCH,AVGCOMPRESSEDROWSIZE,AVGROWCOMPRESSIONRATIO,AVGROWSIZE,PCTROWSCOMPRESSED,LOGINDEXBUILD,CODEPAGE,COLLATIONSCHEMA,COLLATIONNAME,COLLATIONSCHEMA_ORDERBY,COLLATIONNAME_ORDERBY,ENCODING_SCHEME,PCTPAGESSAVED,LAST_REGEN_TIME,SECPOLICYID,PROTECTIONGRANULARITY,AUDITPOLICYID,AUDITPOLICYNAME,AUDITEXCEPTIONENABLED,DEFINER,ONCOMMIT,LOGGED,ONROLLBACK,LASTUSED,CONTROL,TEMPORALTYPE,TABLEORG,EXTENDED_ROW_SIZE,PCTEXTENDEDROWS,REMARKS
0,MONGOONCPD,CUSTOMER_PAYMENT,ADMIN,U,N,N,None,None,None,None,2021-11-19 16:12:36.657536,2021-11-19 16:12:36.657536,2021-11-19 16:12:36.657536,None,3,-32677,-3,-1,-1,-1,-1,-1,-1,-1,-1,None,None,None,0,0,0,0,0,0,0,N,YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY,0,,0,0,N,,None,R,,N,Y,None,N,,F,None,-1,N,999,-1,-1.0,-1,-1.0,None,1208,SYSIBM,BINARY,SYSIBM,BINARY,,-1,2021-11-19 16:12:36.657536,0,,None,None,N,ADMIN,,,,0001-01-01,,N,N,,-1.0,None
1,MONGOONCPD,CUSTOMER_CONTACT,ADMIN,U,N,N,None,None,None,None,2021-11-19 16:12:34.250417,2021-11-19 16:12:34.250417,2021-11-19 16:12:34.250417,None,7,-32680,-3,-1,-1,-1,-1,-1,-1,-1,-1,None,None,None,0,0,0,0,0,0,0,N,YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY,0,,0,0,N,,None,R,,N,Y,None,N,,F,None,-1,N,999,-1,-1.0,-1,-1.0,None,1208,SYSIBM,BINARY,SYSIBM,BINARY,,-1,2021-11-19 16:12:34.250417,0,,None,None,N,ADMIN,,,,0001-01-01,,N,N,,-1.0,None
2,MONGOONCPD,CUSTOMER,ADMIN,U,N,N,None,None,None,None,2021-11-19 16:12:39.343463,2021-11-19 16:12:39.343463,2021-11-19 16:12:39.539884,None,2,-32675,-3,-1,-1,-1,-1,-1,-1,-1,-1,None,None,None,0,0,0,0,0,0,0,N,YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY,0,,0,0,N,,None,R,,N,Y,None,N,,F,None,-1,N,999,-1,-1.0,-1,-1.0,None,1208,SYSIBM,BINARY,SYSIBM,BINARY,,-1,2021-11-19 16:12:39.343463,0,,None,None,N,ADMIN,,,,0001-01-01,,N,N,,-1.0,None
3,MONGOONCPD,CUSTOMER_IDENTITY,ADMIN,U,N,N,None,None,None,None,2021-11-19 16:12:36.966885,2021-11-19 16:12:36.966885,2021-11-19 16:12:36.966885,None,4,-32676,-3,-1,-1,-1,-1,-1,-1,-1,-1,None,None,None,0,0,0,0,0,0,0,N,YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY,0,,0,0,N,,None,R,,N,Y,None,N,,F,None,-1,N,999,-1,-1.0,-1,-1.0,None,1208,SYSIBM,BINARY,SYSIBM,BINARY,,-1,2021-11-19 16:12:36.966885,0,,None,None,N,ADMIN,,,,0001-01-01,,N,N,,-1.0,None


The tables are all connected through the CUSTOMERS_ID field, which is based on the generated `_id` of the main CUSTOMERS colllection. In order to reassemble these tables into a document, we must join them using this unique identifier. An example of the contents of the CUSTOMERS_CONTACT table is shown below.

In [12]:
%sql -a SELECT * FROM MONGOONCPD.CUSTOMER_CONTACT FETCH FIRST 5 ROWS ONLY

,CUSTOMER_ID,CITY,ZIPCODE,EMAIL,PHONE,STREET,STATE
0,619BB982F8E51AC4BB9E72DD,Somerset,15501,Eugene.Brock@snailmail.com,531-341-0361,Washington Avenue,PA
1,619BB982F8E51AC4BB9E72DE,Greenville,29601,Su.Chandler@nada.com,859-223-3389,Elizabeth Street,SC
2,619BB982F8E51AC4BB9E72DF,Helena,59601,Et.Merrill@nada.com,767-335-9433,Buckingham Drive,MT
3,619BB982F8E51AC4BB9E72E0,Twin Falls,83301,Na.Wells@snailmail.com,388-588-8354,Prospect Street,ID
4,619BB982F8E51AC4BB9E72E1,Little Rock,72201,Jam.Lambert@yetii.com,872-384-9838,Elm Street,AR


In [13]:
%sql -a SELECT * FROM MONGOONCPD.CUSTOMER FETCH FIRST 5 ROWS ONLY

,_ID,CUSTOMERID
0,619BB982F8E51AC4BB9E72DD,100000
1,619BB982F8E51AC4BB9E72DE,100001
2,619BB982F8E51AC4BB9E72DF,100002
3,619BB982F8E51AC4BB9E72E0,100003
4,619BB982F8E51AC4BB9E72E1,100004


In [14]:
%sql -a SELECT * FROM MONGOONCPD.CUSTOMER_IDENTITY FETCH FIRST 5 ROWS ONLY

,CUSTOMER_ID,LASTNAME,BIRTHDATE,FIRSTNAME
0,619BB982F8E51AC4BB9E72DD,Brock,1960-10-13,Eugene
1,619BB982F8E51AC4BB9E72DE,Chandler,1971-02-12,Susan
2,619BB982F8E51AC4BB9E72DF,Merrill,1988-08-02,Ethan
3,619BB982F8E51AC4BB9E72E0,Wells,1959-03-11,Nancy
4,619BB982F8E51AC4BB9E72E1,Lambert,1951-02-12,James


One of the tables that is virtualized from Mongo includes customer confidential data; the credit card number that used by the customer to pay account fees. We use the IBM Cloud Pak for Data Goverance and deep data masking in Data Virtualization to ensure that the credit card number is completely masked. The data in source database does not change but the deep governance in Data Virtualization ensures that any appliation or user who access this data doesn't get access to specific card information. However they can still analyze which are the most popular card types. 

In [15]:
%sql SELECT * from MONGOONCPD.CUSTOMER_PAYMENT FETCH FIRST 5 ROWS ONLY

,CUSTOMER_ID,CARD_TYPE,CARD_NO
0,619BB982F8E51AC4BB9E72DD,DYNY,2185-7633-3330-4141
1,619BB982F8E51AC4BB9E72DE,MCCD,5670-7099-6302-1767
2,619BB982F8E51AC4BB9E72DF,VASA,8268-4203-5900-3171
3,619BB982F8E51AC4BB9E72E0,PKUP,9667-1981-6317-6150
4,619BB982F8E51AC4BB9E72E1,PKUP,2596-3346-3361-7128


A full document record is shown in the following SQL statement which joins all of the tables together.

In [16]:
%%sql -a
SELECT C.CUSTOMERID AS CUSTID, 
       CI.FIRSTNAME, CI.LASTNAME, CI.BIRTHDATE,
       CC.CITY, CC.ZIPCODE, CC.EMAIL, CC.PHONE, CC.STREET, CC.STATE,
       CP.CARD_TYPE, CP.CARD_NO
FROM MONGOONCPD.CUSTOMER C, MONGOONCPD.CUSTOMER_CONTACT CC, 
     MONGOONCPD.CUSTOMER_IDENTITY CI, MONGOONCPD.CUSTOMER_PAYMENT CP
WHERE  CC.CUSTOMER_ID = C."_ID" AND
       CI.CUSTOMER_ID = C."_ID" AND
       CP.CUSTOMER_ID = C."_ID"
FETCH FIRST 3 ROWS ONLY

,CUSTID,FIRSTNAME,LASTNAME,BIRTHDATE,CITY,ZIPCODE,EMAIL,PHONE,STREET,STATE,CARD_TYPE,CARD_NO
0,100000,Eugene,Brock,1960-10-13,Somerset,15501,Eugene.Brock@snailmail.com,531-341-0361,Washington Avenue,PA,DYNY,2185-7633-3330-4141
1,100001,Susan,Chandler,1971-02-12,Greenville,29601,Su.Chandler@nada.com,859-223-3389,Elizabeth Street,SC,MCCD,5670-7099-6302-1767
2,100002,Ethan,Merrill,1988-08-02,Helena,59601,Et.Merrill@nada.com,767-335-9433,Buckingham Drive,MT,VASA,8268-4203-5900-3171


In [17]:
%%sql -a
SELECT * FROM MONGO.CUSTOMER
    FETCH FIRST 3 ROWS ONLY

,CUSTID,FIRSTNAME,LASTNAME,BIRTHDATE,CITY,ZIPCODE,EMAIL,PHONE,STREET,STATE,CARD_TYPE,CARD_NO
0,100000,Eugene,Brock,1960-10-13,Somerset,15501,Eugene.Brock@snailmail.com,531-341-0361,Washington Avenue,PA,DYNY,2185-7633-3330-4141
1,100001,Susan,Chandler,1971-02-12,Greenville,29601,Su.Chandler@nada.com,859-223-3389,Elizabeth Street,SC,MCCD,5670-7099-6302-1767
2,100002,Ethan,Merrill,1988-08-02,Helena,59601,Et.Merrill@nada.com,767-335-9433,Buckingham Drive,MT,VASA,8268-4203-5900-3171
